# Cleaning and pre-processing

pamop@nyu.edu April 2023

This file takes the raw data and cleans and processes it for analyses.

- It removes personal subject identifiers and replaces them with arbitrary
  IDs.
- It excludes subjects who have incomplete game data.
- It organizes each session into a single dataframe of game trials played by a
  dyad with one copy of each trial marked accurately.
- It uses the exclusion criteria to filter the metadata and organize by
  arbitrary subject ID.


In [6]:
# imports
import numpy as np
import pandas as pd
import os

import json
import math
import datetime

import gzip
import warnings

warnings.filterwarnings("ignore")

# display all columns of dataframes
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 60)

In [7]:
# for each branch (version) of the experiment we have a separate data file. these are the included versions for the dataset.
fname = [
    "real-all-v105-data.json.gz",
    "real-all-v104-data.json.gz",
    "real-all-v103-data.json.gz",
    "real-all-v102b-data.json.gz",
]
bigdf = pd.DataFrame()

dataframes = []
for file in fname:
    with gzip.open("../data/raw/" + file, "rb") as f:
        data = json.load(f)
        dataframes.append(pd.DataFrame(data))

bigdf = pd.concat(dataframes, ignore_index=True)

# at this point, bigdf is two columns: id and data. the data column is nested dictionaries with all of that participant's data
# across all stages of the experiment (quiz, captcha, task, demographics, etc)
# unpack the data dictionary
bigdf = pd.concat(
    [bigdf.drop(["data"], axis=1), bigdf["data"].apply(pd.Series)], axis=1
)

# expand userdata column so that each row says what session it is
# user_data = pd.concat([bigdf["id"], bigdf["user_data"].apply(pd.Series)], axis=1)
bigdf = pd.concat(
    [bigdf.drop(["user_data"], axis=1), bigdf["user_data"].apply(pd.Series)], axis=1
)  # i want the userdata to be a part of the bigdf

print(bigdf.shape)
display(bigdf.columns)

AttributeError: 'DataFrame' object has no attribute 'append'

In [3]:
# Cut out subjects who never got partnered
bigdf = bigdf.loc[bigdf["partner"] != "NONE"]
bigdf = bigdf.loc[bigdf["partner"].notna()]

print(bigdf.shape)

(750, 34)


In [4]:
# Rename each player by their session and whether they were red or purple
bigdf["agent"] = ""

for sesh in bigdf["session"].unique():
    ids = bigdf.loc[bigdf["session"] == sesh]["id"].unique()
    if len(ids) != 2:
        print(
            "need two subjects per session. drop this session due to incomplete data: "
            + sesh
        )
        bigdf = bigdf.loc[bigdf["session"] != sesh]
    else:
        # Good, two people in this session. name them properly
        if (
            bigdf.loc[bigdf["id"] == ids[0], "joinAt"].item()
            > bigdf.loc[bigdf["id"] == ids[1], "joinAt"].item()
        ):
            bigdf.loc[bigdf["id"] == ids[0], ["agent"]] = "red"
            bigdf.loc[bigdf["id"] == ids[1], ["agent"]] = "purple"
        else:
            bigdf.loc[bigdf["id"] == ids[0], ["agent"]] = "purple"
            bigdf.loc[bigdf["id"] == ids[1], ["agent"]] = "red"

bigdf["subjid"] = (
    bigdf["session"] + bigdf["agent"]
)  # unique string for each player in each session so don't have to use id
bigdf.insert(0, "subjid", bigdf.pop("subjid"))  # put subjid at the front
bigdf = bigdf.drop(["id"], axis=1)  # remove id

print("Currently have: ")
print("N=" + str(len(bigdf["subjid"].unique())))
# Now we tentatively have this many sessions:
print("N sessions: " + str(len(bigdf["session"].unique())))

need two subjects per session. drop this session due to incomplete data: PinIaC22JNyfbX
need two subjects per session. drop this session due to incomplete data: eRC3vSJeJwOEMO
need two subjects per session. drop this session due to incomplete data: hBZigw9g27JWr7
need two subjects per session. drop this session due to incomplete data: se3SBWNpfm8zM5
need two subjects per session. drop this session due to incomplete data: qyWPm3w1kRh2kA
need two subjects per session. drop this session due to incomplete data: BXYfIZA4WsOHsq
need two subjects per session. drop this session due to incomplete data: WBW92Uv9T6gnGv
need two subjects per session. drop this session due to incomplete data: p6Dp8tOHBzIvhk
need two subjects per session. drop this session due to incomplete data: a67fDSdYlyWqpY
need two subjects per session. drop this session due to incomplete data: wiZI2j5kdexg6Q
need two subjects per session. drop this session due to incomplete data: nDbjpgnS3IzD8b
need two subjects per session. d

In [5]:
# Unfortunately, not all of these sessions are usable because many times, one participant leaves early so they don't make it through all 12 games.
# To determine which sessions to drop, we need to melt the gamedata column to take a closer look at how much data is available per session.

# This will take several minutes to run.

# game_data has one row for each subject, and the columns are trials
game_data = pd.concat(
    [bigdf[["subjid", "session"]], bigdf.pop("game_data").apply(pd.Series)], axis=1
)  # bigdf["game_data"].apply(pd.Series)

# melt to get the trial as rows
game_data = pd.melt(
    game_data,
    id_vars=["subjid", "session"],
    value_vars=game_data.columns[3:],
    var_name="trialNum",
    value_name="trialData",
)
game_data = game_data[game_data["trialData"].notna()]

# sort by subjid with trialNum increasing
game_data = (
    game_data.groupby(["subjid", "session"])
    .apply(lambda x: x.sort_values(["trialNum"], ascending=True))
    .reset_index(drop=True)
)

# unpack trialData column
game_data = pd.concat(
    [
        game_data[["subjid", "session", "trialNum"]],
        game_data["trialData"].apply(pd.Series),
    ],
    axis=1,
)
game_data

,subjid,session,trialNum,purpleEnergy,redScore,purpleXloc,redFirst,redBackpackSize,redPoints,redFinished,gameNum,purplePoints,farmBox,timestamp,turnStartTimestamp,redBackpack,redPointsCumulative,purplePointsCumulative,redXloc,redYloc,turnCount,redEnergy,responseTime,eventName,purpleBackpackSize,objectLayer,condition,gameover,purpleBackpack,legalMoves,counterbalance,purpleScore,purpleFinished,decisionMadeTimestamp,target,agent,farmItems,purpleYloc,bonuspoints,bonus
0,3SMEx832vRG5h1purple,3SMEx832vRG5h1,1,100.0,0.0,3.0,False,5.0,NaN,False,0.0,NaN,,1678824668315,1.678825e+12,,0.0,0.0,2.0,15.0,2.0,100.0,1507.0,targetPicked,3.0,Items01,"{""costCond"":""high"",""resourceCond"":""uneven"",""vi...",False,,"purple_none(3,16) Tomato00(7,8) Turnip01(12,13...",10.0,0.0,False,1.678825e+12,Eggplant00,purple,"Tomato00(7,8) Turnip01(12,13) Turnip00(8,7) St...",16.0,NaN,NaN
1,3SMEx832vRG5h1purple,3SMEx832vRG5h1,2,78.0,0.0,12.0,False,5.0,NaN,False,0.0,NaN,,1678824670864,1.678825e+12,,0.0,0.0,2.0,15.0,2.0,100.0,1507.0,objectEncountered,3.0,Items01,"{""costCond"":""high"",""resourceCond"":""uneven"",""vi...",False,"Eggplant00(22,5)",,10.0,0.0,False,1.678825e+12,Eggplant00,purple,"Tomato00(7,8) Turnip01(12,13) Turnip00(8,7) St...",15.0,NaN,NaN
2,3SMEx832vRG5h1purple,3SMEx832vRG5h1,3,78.0,0.0,12.0,False,5.0,NaN,False,0.0,NaN,,1678824672997,1.678825e+12,,0.0,0.0,2.0,15.0,3.0,100.0,2132.0,targetPicked,3.0,Items01,"{""costCond"":""high"",""resourceCond"":""uneven"",""vi...",False,"Eggplant00(22,5)","red_none(2,15) Tomato00(7,8) Turnip01(12,13) T...",10.0,0.0,False,1.678825e+12,Tomato00,red,"Tomato00(7,8) Turnip01(12,13) Turnip00(8,7) St...",15.0,NaN,NaN
3,3SMEx832vRG5h1purple,3SMEx832vRG5h1,4,78.0,0.0,12.0,False,5.0,NaN,False,0.0,NaN,,1678824675796,1.678825e+12,"Tomato00(22,2)",0.0,0.0,7.0,9.0,3.0,76.0,2132.0,objectEncountered,3.0,Items01,"{""costCond"":""high"",""resourceCond"":""uneven"",""vi...",False,"Eggplant00(22,5)",,10.0,0.0,False,1.678825e+12,Tomato00,red,"Tomato00(22,2) Turnip01(12,13) Turnip00(8,7) S...",14.0,NaN,NaN
4,3SMEx832vRG5h1purple,3SMEx832vRG5h1,5,78.0,0.0,12.0,False,5.0,NaN,False,0.0,NaN,,1678824680299,1.678825e+12,"Tomato00(22,2)",0.0,0.0,7.0,9.0,4.0,76.0,4502.0,targetPicked,3.0,Items01,"{""costCond"":""high"",""resourceCond"":""uneven"",""vi...",False,"Eggplant00(22,5)","purple_none(12,14) box(16,5) Turnip01(12,13) T...",10.0,0.0,False,1.678825e+12,Strawberry01,purple,"Tomato00(22,2) Turnip01(12,13) Turnip00(8,7) S...",14.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259783,zRNUjyaiWBgEP0red,zRNUjyaiWBgEP0,368,41.0,3.0,15.0,False,5.0,NaN,False,11.0,NaN,"Strawberry01(22,8) Strawberry00(23,8) Tomato00...",1680024305747,1.680024e+12,"Turnip00(22,2) Eggplant01(23,2) Turnip02(24,2)...",818.0,960.0,9.0,7.0,13.0,66.0,393.0,targetPicked,3.0,Items08,"{""costCond"":""high"",""resourceCond"":""uneven"",""vi...",False,,"red_none(9,7) box(16,5)",3.0,0.0,True,1.680024e+12,box,red,"Eggplant01(23,2) Eggplant00(25,2) Turnip00(22,...",7.0,NaN,NaN
259784,zRNUjyaiWBgEP0red,zRNUjyaiWBgEP0,369,41.0,3.0,15.0,False,5.0,NaN,False,11.0,NaN,"Strawberry01(22,8) Strawberry00(23,8) Tomato00...",1680024307787,1.680024e+12,,818.0,960.0,16.0,6.0,13.0,48.0,393.0,objectEncountered,3.0,Items08,"{""costCond"":""high"",""resourceCond"":""uneven"",""vi...",False,,,3.0,5.0,True,1.680024e+12,box,red,"Eggplant01(23,9) Eggplant00(26,8) Turnip00(24,...",7.0,NaN,NaN
259785,zRNUjyaiWBgEP0red,zRNUjyaiWBgEP0,370,41.0,3.0,15.0,False,5.0,NaN,False,11.0,NaN,"Strawberry01(22,8) Strawberry00(23,8) Tomato00...",1680024308736,1.680024e+12,,818.0,960.0,16.0,7.0,15.0,42.0,393.0,targetPicked,3.0,Items08,"{""costCond"":""high"",""resourceCond"":""uneven"",""vi...",False,,"red_none(16,7)",3.0,5.0,True,1.680024e+12,none,red,"Eggplant01(23,9) Eggplant00(26,8) Turnip00(24,...",7.0,NaN,NaN
259786,zRNUjyaiWBgEP0red,zRNUjyaiWBgEP0,371,41.0,3.0,15.0,False,5.0,126.0,Tru

In [6]:
# Are there are 12 games in each session?
print(str(game_data["session"].nunique()) + " sessions")
print(str(game_data["subjid"].nunique()) + " subjects (data records in smile)")
incomplete = [
    name for name, gp in game_data.groupby(["session"])["gameNum"] if gp.nunique() < 12
]
print(incomplete)

print("number of incomplete sessions: " + str(len(incomplete)))

print("Dropping incomplete sessions... ")
game_data = game_data.loc[~game_data["session"].isin(incomplete)]

print("\nN=" + str(game_data.subjid.nunique()))
print("or " + str(game_data.session.nunique()) + " twelve-game sessions.")

368 sessions
735 subjects (data records in smile)
['3SMEx832vRG5h1', '86ny31f51Kyf8X', '8XRZXj87KnA1dx', '8bkmVMD1WAhASe', 'DH3YDiFAy9blYY', 'HH7fvQQGfSwFyD', 'M9cEyrw2UXKlaG', 'SSzMZUdI5zogmR', 'TJiWEdWHp2OZFv', 'UUvucQBOIgF0tp', 'UxvWmcHR1egm0Q', 'WjqSdCAGrYnNnA', 'YtnI1bg8SNE2jW', 'ZZ0Ic81LHRBvsp', 'ZgfTzR1LU9gsLL', 'Zho6bqfYERQYCf', 'cED3rPb2CC3sSu', 'epStUGMF8gyYUf', 'fYlMwv024bnMa5', 'fkRzaSlWQHKpqw', 'fs3Q7MRKzokPPS', 'i5vzEU82Nf9I3d', 'i7wlRIzR8jhlNm', 'iFYw4xJHXUUeTt', 'j7XaVdZ31zBUhW', 'kCCl6lLdQrYIKV', 'mJfL2XUTu9D8Tr', 'mOS7vIIkM92azZ', 'mObqD3CDzCg1ZG', 'n2BY9lb5w2txPK', 'nAxSKJP80Zlemo', 'nKh4lWQW76ECI8', 'nsRFzukWzCsYgb', 'nwDOyVxc54jsOH', 'o0B0Jiy2Gi1Hf6', 'oFC7DL71CJbe7W', 'orxdhsueCyWbPy', 'qBDf5rejg83qF2', 'ruJI1RISQDUtbP', 'sYel4LjqP4TQeL', 'sb7OrjrSWyQgnJ', 't7OvwtJ2nEgXsB', 'tR80aKa9Y2jy5u', 'tzd3bnuLGsExuV', 'ubi3bAZpuYDV0Q', 'udisLeQJEGwdfO', 'vR8UGhNDc7PKE9', 'vYHZtrnBF562h9', 'x4qx26KJlekuMf', 'xYeqMhrJpSC67B', 'yQ346swk7W0QXf', 'ybRnDx5w9Qh9Pj']
number of inc

In [7]:
# Furthermore, even though these sessions have 12 games (great sign!) there are cases where players quit in the last game.
# in that case, they still have incomplete data (even though they have data for 12 games, the last one is incomplete) so need to be excluded.

df = game_data.loc[game_data["eventName"].isin(["targetPicked", "objectEncountered"])]

gamelengths = {}
incomplete_sessions = []

# for each session for each game, find the length of that game. e.g., turnCount when box is last encountered (when len(farmBox)==len(farmItems))
grouped = df.groupby(["session", "gameNum"])
for name, group in grouped:
    # turn count when game is done
    lastturns = group.loc[group["target"] == "box"].loc[
        group["farmItems"].str.len() == group["farmBox"].str.len()
    ]["turnCount"]

    # are there games where there are multiple values for game length (e.g., got two different values from the two participants?)
    # print(lastturns)
    if len(lastturns) > 2:
        print(
            "err: more than two values for the last turn (should be only two since one per subject)"
        )
        print(lastturns)
        print(name)
        # print(group)
    elif len(lastturns) == 1:
        print(
            "err: only one value for last turn. this prob happened on a final game where only one player's data saved"
        )
        print(name)
        print(lastturns)
        lastturn = lastturns.item()  # lastturns.iloc[0].item()
    elif len(lastturns) == 0:
        print("err: this game did not finish so it is incomplete data!!!")
        incomplete_sessions.append(name[0])
        print(name)
    else:  # two values exactly
        if lastturns.iloc[0].item() == lastturns.iloc[1].item():
            # good! same number as they should be
            lastturn = lastturns.iloc[0].item()
        else:
            print("uh oh, two different values for last turn: ")
            print(lastturns)
            print(name)
            # print(group)
            lastturn = 10000

    # print('last turn: '+str(lastturn))
    gamelengths[name] = lastturn

# gamelengths
print("sessions to drop: ")
print(incomplete_sessions)

print("Dropping incomplete sessions... ")
df = df.loc[~df["session"].isin(incomplete_sessions)]

print("\nN=" + str(df.subjid.nunique()))
print("or " + str(df.session.nunique()) + " FULL AND COMPLETE sessions.")

# We also need to clean up the ends of the games to easily group them
# mark the last trial of the game as such
df["lastTrial"] = df.apply(
    lambda row: (
        True
        if row["turnCount"] == gamelengths[(row["session"], row["gameNum"])]
        else False
    ),
    axis=1,
)

# keep only the trials before the last move of the game
df["keep"] = df.apply(
    lambda row: (
        True
        if row["turnCount"] <= gamelengths[(row["session"], row["gameNum"])]
        else False
    ),
    axis=1,
)
df = df[df["keep"]]

# keep trials with targetPicked only, except also keep lastTrial objectEncountered (that's our new "gameFinished" row)
df["keep"] = df.apply(
    lambda row: (
        False
        if (row["eventName"] == "objectEncountered" and not row["lastTrial"] == True)
        else True
    ),
    axis=1,
)
df = df[df["keep"]]

# mark those rows as the row ending the game
df["gameover"] = df.apply(
    lambda row: True if row["eventName"] == "objectEncountered" else False, axis=1
)

# calculate the points earned for that game
df["redPoints"] = df.apply(
    lambda row: row["redEnergy"] * row["redScore"] if row["gameover"] == True else 0,
    axis=1,
)
df["purplePoints"] = df.apply(
    lambda row: (
        row["purpleEnergy"] * row["purpleScore"] if row["gameover"] == True else 0
    ),
    axis=1,
)

df = df.drop(
    ["purpleFinished", "redFinished", "keep", "bonuspoints", "bonus"], axis=1
)  # dont need these one
df

err: only one value for last turn. this prob happened on a final game where only one player's data saved
('Bj5ET9K9eXGWRE', 11.0)
12432    12.0
Name: turnCount, dtype: float64
err: only one value for last turn. this prob happened on a final game where only one player's data saved
('GIpt06E7scyQAs', 11.0)
20914    12.0
Name: turnCount, dtype: float64
err: only one value for last turn. this prob happened on a final game where only one player's data saved
('LLlh0LYAxsUnae', 11.0)
31024    11.0
Name: turnCount, dtype: float64
err: only one value for last turn. this prob happened on a final game where only one player's data saved
('MnUx0OULOzy46v', 11.0)
34749    11.0
Name: turnCount, dtype: float64
err: this game did not finish so it is incomplete data!!!
('Pv9l6y8BAiNqoH', 11.0)
err: only one value for last turn. this prob happened on a final game where only one player's data saved
('Ujdl2K6GeflUWs', 11.0)
58217    13.0
Name: turnCount, dtype: float64
err: only one value for last turn. th

,subjid,session,trialNum,purpleEnergy,redScore,purpleXloc,redFirst,redBackpackSize,redPoints,gameNum,purplePoints,farmBox,timestamp,turnStartTimestamp,redBackpack,redPointsCumulative,purplePointsCumulative,redXloc,redYloc,turnCount,redEnergy,responseTime,eventName,purpleBackpackSize,objectLayer,condition,gameover,purpleBackpack,legalMoves,counterbalance,purpleScore,decisionMadeTimestamp,target,agent,farmItems,purpleYloc,lastTrial
492,4ISIiFA1J99JScpurple,4ISIiFA1J99JSc,1,100.0,0.0,3.0,True,4.0,0.0,0.0,0.0,,1678818024693,1.678818e+12,,0.0,0.0,2.0,15.0,2.0,100.0,3506.0,targetPicked,4.0,Items04,"{""costCond"":""high"",""resourceCond"":""even"",""visi...",False,,"red_none(2,15) Tomato04(9,13) Turnip02(14,15) ...",12.0,0.0,1.678818e+12,Tomato04,red,"Tomato04(9,13) Turnip02(14,15) Turnip00(11,7) ...",16.0,False
494,4ISIiFA1J99JScpurple,4ISIiFA1J99JSc,3,100.0,0.0,3.0,True,4.0,0.0,0.0,0.0,,1678818030573,1.678818e+12,"Tomato04(22,2)",0.0,0.0,9.0,14.0,3.0,82.0,3786.0,targetPicked,4.0,Items04,"{""costCond"":""high"",""resourceCond"":""even"",""visi...",False,,"purple_none(3,16) Turnip02(14,15) Turnip00(11,...",12.0,0.0,1.678818e+12,Eggplant01,purple,"Tomato04(22,2) Turnip02(14,15) Turnip00(11,7) ...",16.0,False
496,4ISIiFA1J99JScpurple,4ISIiFA1J99JSc,5,84.0,0.0,5.0,True,4.0,0.0,0.0,0.0,,1678818034902,1.678818e+12,"Tomato04(22,2)",0.0,0.0,9.0,13.0,4.0,82.0,2502.0,targetPicked,4.0,Items04,"{""costCond"":""high"",""resourceCond"":""even"",""visi...",False,"Eggplant01(22,5)","red_none(9,13) box(16,5) Turnip02(14,15) Turni...",12.0,0.0,1.678818e+12,Strawberry02,red,"Tomato04(22,2) Turnip02(14,15) Turnip00(11,7) ...",11.0,False
498,4ISIiFA1J99JScpurple,4ISIiFA1J99JSc,7,84.0,0.0,5.0,True,4.0,0.0,0.0,0.0,,1678818036701,1.678818e+12,"Tomato04(22,2) Strawberry02(23,2)",0.0,0.0,9.0,13.0,5.0,80.0,1768.0,targetPicked,4.0,Items04,"{""costCond"":""high"",""resourceCond"":""even"",""visi...",False,"Eggplant01(22,5)","purple_none(5,10) box(16,5) Turnip02(14,15) Tu...",12.0,0.0,1.678818e+12,Turnip01,purple,"Tomato04(22,2) Turnip02(14,15) Turnip00(11,7) ...",10.0,False
500,4ISIiFA1J99JScpurple,4ISIiFA1J99JSc,9,82.0,0.0,5.0,True,4.0,0.0,0.0,0.0,,1678818038234,1.678818e+12,"Tomato04(22,2) Strawberry02(23,2)",0.0,0.0,9.0,14.0,6.0,80.0,1501.0,targetPicked,4.0,Items04,"{""costCond"":""high"",""resourceCond"":""even"",""visi...",False,"Eggplant01(22,5) Turnip01(23,5)","red_none(9,14) box(16,5) Turnip02(14,15) Turni...",12.0,0.0,1.678818e+12,Strawberry01,red,"Tomato04(22,2) Turnip02(14,15) Turnip00(11,7) ...",10.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259778,zRNUjyaiWBgEP0red,zRNUjyaiWBgEP0,363,46.0,3.0,15.0,False,5.0,0.0,11.0,0.0,"Strawberry01(22,8) Strawberry00(23,8) Tomato00...",1680024300039,1.680024e+12,"Turnip00(22,2) Eggplant01(23,2) Turnip02(24,2)...",818.0,960.0,8.0,8.0,10.0,68.0,8002.0,targetPicked,3.0,Items08,"{""costCond"":""high"",""resourceCond"":""uneven"",""vi...",False,,"purple_none(15,7) Turnip01(9,7)",3.0,0.0,1.680024e+12,purplePillow,purple,"Eggplant01(23,2) Eggplant00(25,2) Turnip00(22,...",7.0,False
259780,zRNUjyaiWBgEP0red,zRNUjyaiWBgEP0,365,41.0,3.0,15.0,False,5.0,0.0,11.0,0.0,"Strawberry01(22,8) Strawberry00(23,8) Tomato00...",1680024302833,1.680024e+12,"Turnip00(22,2) Eggplant01(23,2) Turnip02(24,2)...",818.0,960.0,8.0,7.0,11.0,68.0,278.0,targetPicked,3.0,Items08,"{""costCond"":""high"",""resourceCond"":""uneven"",""vi...",False,,"red_none(8,7) box(16,5) Turnip01(9,7)",3.0,0.0,1.680024e+12,Turnip01,red,"Eggplant01(23,2) Eggplant00(25,2) Turnip00(22,...",7.0,False
259782,zRNUjyaiWBgEP0red,zRNUjyaiWBgEP0,367,41.0,3.0,15.0,False,5.0,0.0,11.0,0.0,"Strawberry01(22,8) Strawberry00(23,8) Tomato00...",1680024302854,1.680024e+12,"Turnip00(22,2) Eggplant01(23,2) Turnip02(24,2)...",818.0,960.0,8.0,7.0,12.0,66.0,278.0,targetPicked,3.0,Items08,"{""costCond"":""high"",""resourceCond"":""uneven"",""vi...",False,,"purple_none(15,7)",3.

In [8]:
# I will note that there are some games within a session where we only have one partner's data (this is always the last game of a session
# and I presume it happens because one player finishes too quickly and the smile buffer prevents fast writes to data)

print("cases in which there are not two subjects per game: ")

# another way of confirming the same thing. trying to get those session and game numbers that we need to handle a little differently
# because there is only one subject to use data from.
onesubjectonly = []
for name, grp in df.groupby(["session", "gameNum"]):
    ngameovers = len(grp.loc[grp["gameover"] == True])
    if ngameovers != 2:
        id = grp["subjid"].unique()
        # print(id[0])
        onesubjectonly.append(name)
        # print(name)
        # print(ngameovers)
print(str(len(onesubjectonly)) + " games, " + str(onesubjectonly))

# print(onesubjectonly)

cases in which there are not two subjects per game: 
17 games, [('Bj5ET9K9eXGWRE', 11.0), ('GIpt06E7scyQAs', 11.0), ('LLlh0LYAxsUnae', 11.0), ('MnUx0OULOzy46v', 11.0), ('Ujdl2K6GeflUWs', 11.0), ('Uyet9JuJtlLzZb', 11.0), ('YTLpZxpJ8wFKYn', 11.0), ('bSxiPUDOjnyQuF', 11.0), ('bVmYLCYVLxicfd', 11.0), ('gA6I24mQCRlplv', 11.0), ('hqdXjJ50oSQrNk', 11.0), ('mB2Zp8vYtp5D8Q', 11.0), ('mpcGaIj4HGABbb', 11.0), ('nBkV82AICknxGh', 11.0), ('oGOi0GITrDQ7sp', 11.0), ('qR0OZtlpBD00E3', 11.0), ('utdwJjYg9S6Dx6', 11.0)]


In [9]:
# for each game, keep only the data from one subject (since each subject saves all trials from each game)
# if two subjects data available, make sure you pick one who has complete data (because sometimes one of them doesn't D:)
# if one subject available, that is the one

for name, grp in df.groupby(["session", "gameNum"]):
    sesh = name[0]
    game = name[1]

    if name in onesubjectonly:
        # handle this case - we only have one subject's copy of the data from this game
        # what color is the player we DO have the data from?
        id = grp["subjid"].unique()[0]  # .item()
        color = "red" if "red" in id else "purple" if "purple" in id else "whatdawhat"
        partnercolor = (
            "red"
            if color == "purple"
            else "purple" if color == "red" else "whatdeewhat"
        )

        # find the rows in this game that were choices made by the partner of the subjid player
        mask = (
            (df["session"] == sesh)
            & (df["gameNum"] == game)
            & (df["agent"] == partnercolor)
        )
        # rename the subjid for all the partner's data so it's associated with the correct subjid (e.g., for subj level analysis)
        df.loc[mask, "subjid"] = sesh + partnercolor

# now we are done handling the special cases so everyone left should have two copies of the game data for each game
# so we have two copies of the game data for this game (one from each subject)
# drop trials where subjid and agent color mismatch
mask = ~(
    (df["agent"] == "purple") & (df["subjid"].str.contains("red"))
)  # (df['agent']=='purple') & (df['subjid'].str.contains('purple')) or (df['agent']=='red') & (df['subjid'].str.contains('red'))
df = df.loc[mask]  # keep everyone BUT mask
mask = ~((df["agent"] == "red") & (df["subjid"].str.contains("purple")))
df = df.loc[mask]
# now we only have one copy of each game trial, labeled correctly by subjid as to who performed the action

# sort by trialNum increasing to interleave red and purple's data so it's consecutive within game
df = df.groupby(["session", "gameNum"]).apply(
    lambda x: x.sort_values(["turnCount"], ascending=True)
)  # .sort_values(ascending=True)

# how did we do?
df = df.reset_index(drop=True)

# anythin else to clean up before we export?
# unpack the condition column
conditions = df["condition"].dropna().map(eval).apply(pd.Series)
df = pd.concat([df, conditions], axis=1)

# reorder the columns to look nice
df = df[
    [
        "subjid",
        "session",
        "trialNum",
        "gameNum",
        "costCond",
        "resourceCond",
        "visibilityCond",
        "redFirst",
        "counterbalance",
        "objectLayer",  # details about this game setup
        "eventName",
        "turnCount",
        "agent",
        "target",  # details about this trial specifically
        "turnStartTimestamp",
        "responseTime",
        "decisionMadeTimestamp",
        "timestamp",  # timing information
        "redXloc",
        "redYloc",
        "purpleXloc",
        "purpleYloc",
        "farmItems",
        "farmBox",
        "purpleBackpack",
        "redBackpack",  # what the game looks like on this trial to both players
        "gameover",
        "legalMoves",  # whether the game is over, what legal moves each player could take
        "purpleBackpackSize",
        "purpleEnergy",
        "purpleScore",
        "purplePoints",
        "purplePointsCumulative",  # purple's current items, energy, and scores
        "redBackpackSize",
        "redEnergy",
        "redScore",
        "redPoints",
        "redPointsCumulative",  # red's current items, energy, and scores
        "lastTrial",  # we added this one to keep track of last trial
    ]
]

# i would like purplePointsCumulative and redPC to update on the gameover trial rather than the first trial of the next game
df.loc[df["gameover"] == True, "purplePointsCumulative"] = (
    df.loc[df["gameover"] == True, "purplePointsCumulative"]
    + df.loc[df["gameover"] == True, "purplePoints"]
)
df.loc[df["gameover"] == True, "redPointsCumulative"] = (
    df.loc[df["gameover"] == True, "redPointsCumulative"]
    + df.loc[df["gameover"] == True, "redPoints"]
)

# how does timestamp differ from turnStartTimestamp and decisionMadeTimestamp?
# "timestamp" is the time when the data was saved to smile. let's rename it for clarity.
df = df.rename(columns={"timestamp": "dataSavedTimestamp"})

# integer columns
df["trialNum"].astype("int")
df = df.astype(
    {
        "trialNum": "int",
        "gameNum": "int",
        "turnCount": "int",
        "responseTime": "int",
        "turnStartTimestamp": "int",
        "decisionMadeTimestamp": "int",
        "dataSavedTimestamp": "int",
        "redXloc": "int",
        "redYloc": "int",
        "purpleXloc": "int",
        "purpleYloc": "int",
        "purpleBackpackSize": "int",
        "purpleEnergy": "int",
        "purpleScore": "int",
        "purplePoints": "int",
        "purplePointsCumulative": "int",
        "redBackpackSize": "int",
        "redEnergy": "int",
        "redScore": "int",
        "redPoints": "int",
        "redPointsCumulative": "int",
    }
)

# booleans
df = df.astype({"gameover": "boolean", "lastTrial": "boolean", "redFirst": "boolean"})

# categorical
df = df.astype(
    {
        "subjid": "category",
        "session": "category",
        "costCond": "category",
        "resourceCond": "category",
        "visibilityCond": "category",
        "counterbalance": "category",
        "objectLayer": "category",
        "eventName": "category",
        "agent": "category",
        "target": "category",
    }
)

In [10]:
# We have now fully processed the gamedata. In doing so, we determined which sessions to keep and which to exclude. We cleaned the data and removed redundant copies.
include_sessions = df["session"].unique()
include_subjects = df["subjid"].unique()

print("Num sessions: " + str(len(include_sessions)))
print("N participants: " + str(len(include_subjects)))

Num sessions: 314
N participants: 628


In [11]:
# Let's export the game data to one entire big file and then as separate csv for each session.

# save this gamedata save all in one
path = "../data/alldata"
os.makedirs(path, exist_ok=True)

write_to_file = True


if write_to_file:

    print("writing " + str(df["session"].nunique()) + " files, one per session")
    for name, grp in df.groupby(["session"]):
        fname = path + "/gamedata_" + name + ".csv"
        grp.to_csv(fname, index=False)
        print(".", end=" ")
        # print('file saved to: '+ fname)
    print("\ndone!")

    fname = path + "/gamedata_all" + ".csv"
    df.to_csv(fname, index=False)
    print("file saved to: " + fname)

    # save a list of the valid sessions in case you need to grab them from the raw uncleaned data later
    fname = path + "/valid_sessions" + ".csv"
    np.savetxt(fname, df["session"].unique(), delimiter=",", fmt="%s")
    print("file saved to: " + fname)

writing 314 files, one per session
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
done!
file saved to: ../data/alldata/gamedata_all.csv
file saved to: ../data/alldata/valid_sessions.csv


In [12]:
# Let's filter the rest of the dataset (e.g., demographics, quiz, captcha) by the included subjects only.
# Then we can save subject level data for these other parts of the experiment if we want to explore them and relate them to the game data.
bigdf = bigdf.loc[bigdf["session"].isin(df["session"])]
bigdf

,subjid,smileconfig,captcha_data,quiz_form,counterbalance,game_setup,bonus_points,time_data,started_game,demographic_form,bonus_amount,consented,conditions,mode,finalsurvey_data,issues,withdraw,quiz_attempts,recruitment_service,recruitment_info,datetime,browser_data,status,trial_num,done,withdraw_data,playerId,partner,session,joinAt,partnerName,playerName,status,agent
1,d2YvnIb1SzT5XLred,"{'min_write_interval': '2000', 'auto_save': 't...","[{'dragY': 0, 'event': 'dragend', 'timestamp':...",[{'goal': 'When all the vegetables are in the ...,0,{},759,[],True,"{'country': 'United States', 'normal_vision': ...",0.76,True,"{'resourceCond': 'even', 'visibilityCond': 'se...",production,{},[],False,1,prolific,"{'session_id': '642c709d64877106e6c4388f', 'st...",2023-04-04T18:47:03.287Z,"[{'timestamp': {'seconds': 1680634023, 'nanose...",endFinished,0,True,{},1SzT5XLyZefoThroWYjy,d2YvnIbNKnEc0MgWd7Sc,d2YvnIb1SzT5XL,1.680634e+12,Yvn,zT5,play,red
3,GaoxGQc1Xmby2bred,"{'services_allowed': 'amt,prolific,sona,web', ...","[{'dragX': 12, 'event': 'dragend', 'timestamp'...",[{'bonus': 'Number of red veggies in the box t...,1,{},1295,[],True,"{'color_blind': 'No', 'fluent_english': 'Yes',...",1.30,True,"{'resourceCond': 'uneven', 'costCond': 'high',...",production,{},[],False,1,prolific,"{'prolific_id': '5c2a92b4b8e5f900015b3f58', 's...",2023-04-04T18:46:44.851Z,"[{'timestamp': {'seconds': 1680634070, 'nanose...",endFinished,0,True,{},1Xmby2bWFQ8qq4F44DS9,GaoxGQcfiE7Nnu7Nm0df,GaoxGQc1Xmby2b,1.680634e+12,oxG,mby,play,red
4,TivIAtz1sWuRopred,"{'min_write_interval': '2000', 'mode': 'produc...","[{'dragY': 8, 'dragX': 9, 'event': 'dragend', ...","[{'attempt': 1, 'cent': '1', 'bonus': 'Number ...",4,{},269,[],True,"{'gender': 'Male', 'hispanic': 'No', 'dob': '1...",0.27,True,"{'resourceCond': 'uneven', 'visibilityCond': '...",production,{},[],False,2,prolific,"{'session_id': '64231f31804c737b91b0f2f7', 'pr...",2023-03-28T17:10:04.714Z,"[{'timestamp': {'seconds': 1680024076, 'nanose...",endFinished,0,True,{},1sWuRopCJyArujSEkN7e,TivIAtzSuCe17yCg6VsJ,TivIAtz1sWuRop,1.680025e+12,vIA,WuR,play,red
7,Z7SpYO22HIBbPMpurple,{'project_ref': 'pamop-computational_helping-v...,"[{'dragX': -7, 'timestamp': 1680622211664, 'dr...",[{'bonus': 'Number of red veggies in the box t...,5,{},2148,[],True,"{'household_income': '$60,000–$79,999', 'hispa...",2.15,True,"{'visibilityCond': 'self', 'costCond': 'low', ...",production,{},[],False,1,prolific,"{'session_id': '642c41e650cb8379521d5518', 'pr...",2023-04-04T15:27:39.036Z,"[{'timestamp': {'seconds': 1680622059, 'nanose...",endFinished,0,True,{},2HIBbPMuiZdUXX7zj8jL,Z7SpYO2pWD5Vqay3zFXU,Z7SpYO22HIBbPM,1.680622e+12,SpY,IBb,play,purple
10,RophlMz2pnT4Tjpurple,"{'windowsizer_request': {'height': '600px', 'w...","[{'event': 'dragend', 'dragX': -1, 'dragY': 12...","[{'move': 'Mouse clicks', 'pass': 'Click on th...",15,{},750,[],True,{'education_level': 'Technical/Community Colle...,0.75,True,"{'visibilityCond': 'self', 'resourceCond': 'ev...",production,{},[],False,2,prolific,"{'study_id': '642205d9c8ef5f39930632b9', 'sess...",2023-03-28T15:42:48.088Z,[],endFinished,0,True,{},2pnT4TjpWyJkmgJAMakg,RophlMzmhqZIqRqK4eQl,RophlMz2pnT4Tj,1.680019e+12,phl,nT4,play,purple
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,w2ixw51E2kOOXppurple,"{'browser_exclude': 'ie', 'code_name': 'patien...","[{'dragY': 2, 'event': 'dragend', 'timestamp':...","[{'move': 'Mouse clicks', 'bonus': 'Number of ...",6,{},3180,[],True,"{'dob': '1988-04-06', 'education_level': 'Unde...",3.18,True,"{'costCond': 'low', 'resourceCond': 'even', 'v...",production,{},[],False,3,prolific,"{'session_id': '640a4710fae687207b478683', 'st...",2023-03-09T20:52:56.153Z,"[{'timestamp': {'seconds': 1678395176, 'nanose...",NaN,0,True,{},w2ixw51ETMVQaBTvvSNg,E2kOOXpl8eJvxmDXVSJw,w2ixw51E2kOOXp,1.678396e+12,kOO,ixw,play,purple
126,yExSMPVAyBy9Yipurple,"{'google_analytics_id': 'G-DXYD

In [13]:
# expand columns of dictionaries and drop original column
smileconfig = pd.concat(
    [bigdf["subjid"], bigdf.pop("smileconfig").apply(pd.Series)], axis=1
)
captcha_data = pd.concat(
    [bigdf["subjid"], bigdf.pop("captcha_data").apply(pd.Series)], axis=1
)
quiz_form = pd.concat(
    [bigdf["subjid"], bigdf.pop("quiz_form").apply(pd.Series)], axis=1
)
demographic_form = pd.concat(
    [bigdf["subjid"], bigdf.pop("demographic_form").apply(pd.Series)], axis=1
)
conditions = pd.concat(
    [bigdf["subjid"], bigdf.pop("conditions").apply(pd.Series)], axis=1
)
recruitment_info = pd.concat(
    [bigdf["subjid"], bigdf.pop("recruitment_info").apply(pd.Series)], axis=1
)
browser_data = pd.concat(
    [bigdf["subjid"], bigdf.pop("browser_data").apply(pd.Series)], axis=1
)

bigdf

,subjid,counterbalance,game_setup,bonus_points,time_data,started_game,bonus_amount,consented,mode,finalsurvey_data,issues,withdraw,quiz_attempts,recruitment_service,datetime,status,trial_num,done,withdraw_data,playerId,partner,session,joinAt,partnerName,playerName,status,agent
1,d2YvnIb1SzT5XLred,0,{},759,[],True,0.76,True,production,{},[],False,1,prolific,2023-04-04T18:47:03.287Z,endFinished,0,True,{},1SzT5XLyZefoThroWYjy,d2YvnIbNKnEc0MgWd7Sc,d2YvnIb1SzT5XL,1.680634e+12,Yvn,zT5,play,red
3,GaoxGQc1Xmby2bred,1,{},1295,[],True,1.30,True,production,{},[],False,1,prolific,2023-04-04T18:46:44.851Z,endFinished,0,True,{},1Xmby2bWFQ8qq4F44DS9,GaoxGQcfiE7Nnu7Nm0df,GaoxGQc1Xmby2b,1.680634e+12,oxG,mby,play,red
4,TivIAtz1sWuRopred,4,{},269,[],True,0.27,True,production,{},[],False,2,prolific,2023-03-28T17:10:04.714Z,endFinished,0,True,{},1sWuRopCJyArujSEkN7e,TivIAtzSuCe17yCg6VsJ,TivIAtz1sWuRop,1.680025e+12,vIA,WuR,play,red
7,Z7SpYO22HIBbPMpurple,5,{},2148,[],True,2.15,True,production,{},[],False,1,prolific,2023-04-04T15:27:39.036Z,endFinished,0,True,{},2HIBbPMuiZdUXX7zj8jL,Z7SpYO2pWD5Vqay3zFXU,Z7SpYO22HIBbPM,1.680622e+12,SpY,IBb,play,purple
10,RophlMz2pnT4Tjpurple,15,{},750,[],True,0.75,True,production,{},[],False,2,prolific,2023-03-28T15:42:48.088Z,endFinished,0,True,{},2pnT4TjpWyJkmgJAMakg,RophlMzmhqZIqRqK4eQl,RophlMz2pnT4Tj,1.680019e+12,phl,nT4,play,purple
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,w2ixw51E2kOOXppurple,6,{},3180,[],True,3.18,True,production,{},[],False,3,prolific,2023-03-09T20:52:56.153Z,NaN,0,True,{},w2ixw51ETMVQaBTvvSNg,E2kOOXpl8eJvxmDXVSJw,w2ixw51E2kOOXp,1.678396e+12,kOO,ixw,play,purple
126,yExSMPVAyBy9Yipurple,4,{},422,[],True,0.43,True,production,{},[],False,1,prolific,2023-03-08T22:33:57.653Z,NaN,0,True,{},yExSMPVN9tcjP3NnYGhG,AyBy9Yi8zdH5QerDLMOj,yExSMPVAyBy9Yi,1.678315e+12,By9,xSM,play,purple
127,yM7Mq6iCXeB9Svpurple,5,{},3030,[],True,3.03,True,production,{},[],False,1,prolific,2023-03-08T22:37:55.253Z,NaN,0,False,{},yM7Mq6i512GasHQXDkVP,CXeB9SvCqcbYA185YwuN,yM7Mq6iCXeB9Sv,1.678315e+12,eB9,7Mq,play,purple
128,yPQzz9iPJVUCQmred,12,{},1090,[],True,1.09,True,production,{},[],False,2,prolific,2023-03-08T21:28:37.338Z,NaN,0,True,{},yPQzz9iQt3T6FxZaWDZm,PJVUCQmybYYnpFzpjNXq,yPQzz9iPJVUCQm,1.678312e+12,VUC,Qzz,play,red


In [14]:
fnamedict = {
    "smileconfig_data": smileconfig,
    "captcha_data": captcha_data,
    "quiz_data": quiz_form,
    "demographic_data": demographic_form,
    "conditions_data": conditions,
    "recruitment_data": recruitment_info,
    "browser_data": browser_data,
    "misc_data": bigdf,
}

for f, df in fnamedict.items():
    fname = path + "/" + f + ".csv"
    df.to_csv(fname, index=False)
    print("file saved to: " + fname)

file saved to: ../data/alldata/smileconfig_data.csv
file saved to: ../data/alldata/captcha_data.csv
file saved to: ../data/alldata/quiz_data.csv
file saved to: ../data/alldata/demographic_data.csv
file saved to: ../data/alldata/conditions_data.csv
file saved to: ../data/alldata/recruitment_data.csv
file saved to: ../data/alldata/browser_data.csv
file saved to: ../data/alldata/misc_data.csv
